# Hyperbolic discount function: test hypotheses in terms of $\log(k)$

This is the second part of our core analyis. Here we quantitatively evaluate the hypotheses about how discounting for the various commodities will change going from fasting to control condition. We will do this by using maximum likeligood parameter estimation for each of the hypotheses, and then compute the $\Delta AIC$ and $\Delta BIC$ metrics to compare the goodness of the models, controling for model complexity (number of parameters).

Burnham & Anderson (2002) present heuristics to interpret the $\Delta$ AIC.
- $\Delta AIC = 0-2$: litle to distiniguish the models
- $\Delta AIC = 4-7$: considerably less support for the model with higher AIC
- $\Delta AIC > 10$: essentially no support for the model with higher AIC

Kas et al (1995) present similar heuristics to interpret the $\Delta$ BIC.
- $\Delta BIC = 0-2$: litle to distiniguish the models
- $\Delta BIC = 2-6$: positive evidence against the model with higher BIC
- $\Delta BIC = 6-10$: strong evidence against the model with higher BIC
- $\Delta BIC > 10$: very strong evidence against the model with higher BIC

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import cauchy
from scipy.optimize import minimize

In [2]:
def long_to_wide(df, target_param):
    '''Convert long-form data to wide-form data'''
    # pivot long to wide
    df = df.set_index('id')
    df = df.pivot_table(index='id', columns=['commodity', 'condition'], values=target_param)
    # collapse column multi-index 
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    # set index to a column
    df = df.reset_index()
    return df

data_long = pd.read_csv('parameter_estimation_Hyperbolic.csv')
data_wide = long_to_wide(data_long, target_param='logk')

delta_logk_food = (data_wide['food F'] - data_wide['food C']).values
delta_logk_money = (data_wide['money F'] - data_wide['money C']).values
delta_logk_music = (data_wide['music F'] - data_wide['music C']).values

Create data. Must be in this format as a dict with fields `'delta_food'`, `'delta_money'`, `'delta_music'`.

In [3]:
data = {'delta_food': delta_logk_food,
        'delta_money': delta_logk_money, 
        'delta_music': delta_logk_music}

Define our models. We must do it here, where `data` is in the local scope, in order to call the `nll` method as a static method in order to use `scipy.optimize.minimize`.

In [4]:
n_observations = len(data['delta_food'])


class Model:

    def calc_aic(self):
        return -2*self.ll + 2*self.free_params

    def calc_bic(self):
        return -2*self.ll + np.log(n_observations)*self.free_params

    def fit(self):
        '''Find parameters which minimise the negative log likelihood.'''
        if not self.bounds:
            result = minimize(self.nll, self.x0,
                              method='Nelder-Mead',
                              options={'disp': True})
        else:
            result = minimize(self.nll, self.x0,
                              method='L-BFGS-B', bounds=self.bounds,
                              options={'disp': True})
        self.mlparams = result.x
        self.nll = result.fun
        self.ll = -self.nll
        self.AIC = self.calc_aic()
        self.BIC = self.calc_bic()
        return self


class H1(Model):
    """Our control (trait-only) model which assumes zero change in AUC"""

    name = "1. Trait only"
    x0 = [0.05]
    free_params = len(x0)
    bounds = None

    @staticmethod
    def nll(params):
        return (-sum(cauchy.logpdf(data['delta_food'], loc=0, scale=params[0]) +
                     cauchy.logpdf(data['delta_money'], loc=0, scale=params[0]) +
                     cauchy.logpdf(data['delta_music'], loc=0, scale=params[0])))


class H2(Model):
    """In-domain model"""

    name = "2. In-domain"
    x0 = [+0.25, 0.05]
    free_params = len(x0)
    bounds = None

    @staticmethod
    def nll(params):
        return -sum(cauchy.logpdf(data['delta_food'], loc=params[0], scale=params[1]) +
                   cauchy.logpdf(data['delta_money'], loc=0, scale=params[1]) +
                   cauchy.logpdf(data['delta_music'], loc=0, scale=params[1]))


class H3(Model):
    """Monetary primacy model"""

    name = "3. Monetary primacy"
    x0 = [0.25, 0.05]
    free_params = len(x0)
    bounds = None

    @staticmethod
    def nll(params):
        return -sum(cauchy.logpdf(data['delta_food'], loc=params[0], scale=params[1]) +
               cauchy.logpdf(data['delta_money'], loc=params[0], scale=params[1]) +
               cauchy.logpdf(data['delta_music'], loc=0, scale=params[1]))


class H4(Model):
    """Devaluation model"""

    name = "4. Devaluation"
    x0 = [0.25, -0.1, 0.05]
    free_params = len(x0)
    bounds = [(0., None), (None, 0.), (0., None)]

    @staticmethod
    def nll(params):
        return -sum(cauchy.logpdf(data['delta_food'], loc=params[0], scale=params[2]) +
               cauchy.logpdf(data['delta_money'], loc=params[1], scale=params[2]) +
               cauchy.logpdf(data['delta_music'], loc=params[1], scale=params[2]))


class H5(Model):
    """Spillover model"""

    name = "5. Spillover"
    x0 = [+0.25, +0.1, 0.05]
    free_params = len(x0)
    bounds = None

    @staticmethod
    def nll(params):
        return -sum(cauchy.logpdf(data['delta_food'], loc=params[0], scale=params[2]) +
               cauchy.logpdf(data['delta_money'], loc=params[1], scale=params[2]) +
               cauchy.logpdf(data['delta_music'], loc=params[1], scale=params[2]))


class H6(Model):
    """State-only model"""

    name = "6. State-only"
    x0 = [+0.25, 0.05]
    free_params = len(x0)
    bounds = None

    @staticmethod
    def nll(params):
        return -sum(cauchy.logpdf(data['delta_food'], loc=params[0], scale=params[1]) +
               cauchy.logpdf(data['delta_money'], loc=params[0], scale=params[1]) +
               cauchy.logpdf(data['delta_music'], loc=params[0], scale=params[1]))

Create and fit the models.

In [5]:
models = [H1(), H2(), H3(), H4(), H5(), H6()]

models = [model.fit() for model in models]

for model in models:
    print(f'model: {model.name}, params: {model.mlparams}')

Optimization terminated successfully.
         Current function value: 365.664764
         Iterations: 21
         Function evaluations: 42
Optimization terminated successfully.
         Current function value: 347.372101
         Iterations: 52
         Function evaluations: 98
Optimization terminated successfully.
         Current function value: 349.159253
         Iterations: 54
         Function evaluations: 105
Optimization terminated successfully.
         Current function value: 340.165061
         Iterations: 125
         Function evaluations: 218
Optimization terminated successfully.
         Current function value: 348.443621
         Iterations: 46
         Function evaluations: 89
model: 1. Trait only, params: [1.0959375]
model: 2. In-domain, params: [1.86257004 1.03288556]
model: 3. Monetary primacy, params: [0.93814007 1.05183251]
model: 4. Devaluation, params: [1.86258682 0.         1.03287569]
model: 5. Spillover, params: [1.84629129 0.46764303 0.99219284]
model: 6. St

Create and save a useful table comparing the hypotheses.

$$
W(IC_m) = \frac{-0.5 \Delta IC_m}{\Sigma_i(-0.5 \Delta IC_i)}
$$


In [6]:
def W(x):
    return np.exp(-x/2)

# summarise data in a DataFrame
aic = np.array([model.AIC for model in models])
delta_aic = aic - min(aic)
waic = W(delta_aic)/sum(W(delta_aic))

bic = np.array([model.BIC for model in models])
delta_bic = bic - min(bic)
wbic = W(delta_bic)/sum(W(delta_bic))

In [7]:
info = {'model': [model.name for model in models],
        'n': [model.free_params for model in models],
        'LL': [model.ll for model in models],
        'AIC': [model.AIC for model in models],
        'deltaAIC': delta_aic,
        'wAIC': waic,
        'BIC': [model.BIC for model in models],
        'deltaBIC': delta_bic,
        'wBIC': wbic,
        'parameters': [model.mlparams for model in models]}

results = pd.DataFrame.from_dict(info)
results.to_csv(f'model_comparison_LOGK_hyperbolic.csv', index=False)
results

,model,n,LL,AIC,deltaAIC,wAIC,BIC,deltaBIC,wBIC,parameters
0,1. Trait only,1,-365.664764,733.329528,46.999405,6.202524e-11,735.241551,43.175359,4.176827e-10,[1.0959375000000042]
1,2. In-domain,2,-347.372101,698.744202,12.414079,2.007598e-03,702.568248,10.502056,5.197137e-03,"[1.8625700367074023, 1.0328855583123948]"
2,3. Monetary primacy,2,-349.159253,702.318507,15.988384,3.361447e-04,706.142553,14.076361,8.701894e-04,"[0.9381400651419424, 1.0518325072548151]"
3,4. Devaluation,3,-347.372101,700.744202,14.414079,7.385539e-04,706.480271,14.414079,7.349862e-04,"[1.8625868160176355, 0.0, 1.0328756873709777]"
4,5. Spillover,3,-340.165061,686.330123,0.000000,9.962301e-01,692.066192,0.000000,9.914177e-01,"[1.8462912894412344, 0.46764302923730405, 0.99..."
5,6. State-only,2,-348.443621,700.887242,14.557119,6.875772e-04,704.711288,12.645096,1.779955e-03,"[0.701001648837808, 1.0338378386801237]"


According to the AIC and BIC metrics:
- the best model is the `spillover` hypothesis. 
- there is essentially no support for any of the other hypotheses.

## References

Burnham, K. P., & Anderson, D. R. (2004). Multimodel Inference: Understanding AIC and BIC in Model Selection, 33(2), 261–304. http://doi.org/10.1177/0049124104268644

Kass, Robert E.; Raftery, Adrian E. (1995), "Bayes Factors", Journal of the American Statistical Association, 90 (430): 773–795. http://doi.org/10.2307/2291091